In [37]:
import torch
import torchvision
import torchvision.models as models
from PIL import Image, ImageFile
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import gc; gc.enable()
import pandas as pd
from torch.utils.data import Dataset
import tensorflow as tf
from tensorflow import keras
from torchvision import transforms

In [38]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
# keras imports
from keras.models import Model
from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Input
# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time

In [39]:
train_dir = '/Users/flatironschool/Documents/Kaggle/Kaggle-DR-detection/data/train/'

In [40]:
json_file = open('Xception.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
xception = keras.models.model_from_json(loaded_model_json)
# load weights into new model
xception.load_weights('Xception.h5') 

In [41]:
datagen = keras.preprocessing.image.ImageDataGenerator()
train_generator = datagen.flow_from_directory(train_dir, target_size = (224,224), classes = ['0','1','2','3','4'],class_mode ='categorical') 
batch_size = 32

Found 21972 images belonging to 5 classes.


In [42]:
get_last_pretrained_layer = keras.backend.function([xception.layers[0].input],
                                  [xception.layers[131].output])

In [47]:
gc.collect()

132

In [52]:
for cycle in range(5):
    train_features = np.zeros(shape=(21972//5, 7, 7, 2048))
    train_labels = np.zeros(shape=(21972//5,5))
    i = 0
    for inputs_batch, labels_batch in train_generator:
        features_batch = get_last_pretrained_layer(inputs_batch)[0]
        train_features[i : i + len(inputs_batch)] = features_batch
        train_labels[i : i +len(inputs_batch)] = labels_batch
        i += len(inputs_batch)
        print(i)
        if i+len(inputs_batch) > train_features.shape[0]:
            break
    train_features = np.reshape(train_features, (21972//5, 7 * 7 * 2048))
    train = pd.DataFrame(train_features)
    trainf = 'train_features_'+str(cycle)+'.csv'
    train.to_csv(trainf)
    bleh, cats = np.where(train_labels)
    y_train = pd.DataFrame(cats)
    y_train_f = 'y_train_'+str(cycle)+'.csv'
    y_train.to_csv(y_train_f)
    del train_features, train_labels, train, y_train, bleh, cats; gc.collect()

32
64
96
128
160
192
224
256
288
320
352
384
416
448
480
512
544
576
608
640
672
704
736
768
800
832
864
896
928
960
992
1024
1056
1088
1120
1152
1184
1216
1248
1280
1312
1344
1376
1408
1440
1472
1504
1536
1568
1600
1632
1664
1696
1728
1760
1792
1824
1856
1888
1920
1952
1984
2016
2048
2080
2112
2144
2176
2208
2240
2272
2304
2336
2368
2400
2432
2464
2496
2528
2560
2592
2624
2656
2688
2720
2752
2784
2816
2848
2880
2912
2944
2976
3008
3040
3072
3104
3136
3168
3200
3232
3264
3296
3328
3360
3392
3424
3456
3488
3520
3552
3584
3616
3648
3680
3712
3744
3776
3808
3840
3872
3904
3936
3968
4000
4032
4064
4096
4128
4160
4192
4224
4256
4288
4320
4352
4384
32
64
96
128
160
192
224
256
288
320
352
384
416
448
480
512
544
576
608
640
672
704
736
768
800
832
864
896
928
960
992
1024
1056
1088
1120
1152
1184
1216
1248
1280
1312
1344
1376
1408
1440
1472
1504
1536
1568
1600
1632
1664
1696
1728
1760
1792
1824
1856
1888
1920
1952
1984
2016
2048
2080
2112
2144
2176
2208
2240
2272
2304
2336
2368
2400
2432
246

In [27]:
del xception, train_generator; gc.collect()

543

In [28]:
from sklearn.model_selection import StratifiedKFold, KFold
import xgboost as xgb
params = {
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 5,
}

In [ ]:

train_cols = train.columns

In [ ]:
import xgbfir

n_fold = 5
folds = KFold(n_fold, shuffle=False, random_state=11)
oof_preds = np.zeros(len(train_features))
# sub_preds = np.zeros(test_.shape[0])

feature_importance_df = pd.DataFrame()

In [5]:
from sklearn.metrics import confusion_matrix, cohen_kappa_score
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train, y_train)):
    
    trn_x, trn_y = train[train_cols].iloc[trn_idx], y_train.iloc[trn_idx]
    val_x, val_y = train[train_cols].iloc[val_idx], y_train.iloc[val_idx]
    gc.collect()
    dtrain = xgb.DMatrix(trn_x, trn_y)
    dval = xgb.DMatrix(val_x, val_y)
    gc.collect()
    
    clf = xgb.train(params=params, dtrain=dtrain, num_boost_round=1500, early_stopping_rounds=100) 
    gc.collect()
    
    oof_preds[val_idx] = clf.predict(xgb.DMatrix(val_x))
#     sub_preds += clf.predict(xgb.DMatrix(test[train_cols])) / folds.n_splits
    gc.collect()
    
    xgbfir.saveXgbFI(clf, feature_names=trn_x.columns, OutputXlsxFile='ieee_xgbfir_%sFold.xlsx'%str(n_fold+1), MaxInteractionDepth=9, MaxHistograms=15)
    gc.collect()
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = pd.DataFrame.from_dict(data=clf.get_fscore(), orient="index", columns=["FScore"])["FScore"].index
    fold_importance_df["fscore"] = pd.DataFrame.from_dict(data=clf.get_fscore(), orient="index", columns=["FScore"])["FScore"].values
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    gc.collect()
    print(cohen_kappa_score(y_test, y_pred))
    print('\nFold %2d cohens kappa %.6f & std %.6f' %(n_fold + 1, cohen_kappa_score(val_y, oof_preds[val_idx]), np.std([oof_preds[val_idx]])))
    gc.collect()

print('\nCV cohens kappa score score %.6f & std %.6f' % (cohen_kappa_score(y_train, oof_preds), np.std((oof_preds))))


NameError: name 'folds' is not defined

In [1]:
X = feat_train
bleh, y = np.where(labels_train)

xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(X, y)

y_pred = xgb_model.predict(feat_test)

NameError: name 'feat_train' is not defined

In [ ]:

bleh, y_test = np.where(labels_test)
print(cohen_kappa_score(y_test, y_pred))